In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
!cp '/gdrive/My Drive/road2/boxes_txt.zip' '/content/boxes_txt.zip'
!cp '/gdrive/My Drive/road2/depth_npy.zip' '/content/depth_npy.zip'

In [0]:
import zipfile

zf = zipfile.ZipFile("/content/boxes_txt.zip", "r")
for i in zf.namelist():
    zf.extract(i, path="/content/boxes_txt_coco/")

zf = zipfile.ZipFile("/content/depth_npy.zip", "r")
for i in zf.namelist():
    zf.extract(i, path="/content/depth_npy/")


mkdir: cannot create directory ‘detections_kitti_system’: Operation not supported


In [0]:
%cd ..
%cd /content/
!mkdir detections_kitti_system

/
/content


In [0]:
def transf_point(matriz, punto):
  import numpy as np
  x = int(np.dot(matriz[0], np.append(punto, [1])) / np.dot(matriz[2], np.append(punto, [1])))
  y = int(np.dot(matriz[1], np.append(punto, [1])) / np.dot(matriz[2], np.append(punto, [1])))
  return [x, y]

def get_coords(box):
  top_left = [box[0], box[1]]
  top_right = [box[2], box[1]]
  bottom_right = [box[2], box[3]]
  bottom_left = [box[0], box[3]] 
  return [top_left, top_right, bottom_right, bottom_left]

def get_center(coord):
  return int(coord[3][0] + ((coord[2][0] - coord[3][0]) / 2))

def get_pixel_vector(matriz, obj):
  """
  Input: 
   - Matriz de transformación.
   - Información de la detección
  Ooutput:
   - Vector (coordenada) transformado del centro-bajo del objeto.
  """
  # Interpretar información del txt (4 números) y devolver coordenadas (4 pares [x,y])
  coords = get_coords(obj)
  # Obtener la coordenada del centro-abajo de la caja
  x_center = get_center(coords)
  # Calcular y devolver nueva coordenada en el plano BEV
  return transf_point(matriz, [x_center, coords[3][1]])

def get_depth_vector(center_coord, cm2px, dimensions):
  """
  Input: 
   - Vector (coordenada) transformado del centro-bajo del objeto.
   - Vector de relación centímetro-pixel.
   - Dimensiones de la imagen.
  Ooutput:
   - Vector de profundidad de la coordenada centro-bajo del objeto.
  """
  # Calcular distancia en el eje X
  x_pixels = center_coord[0] - (dimensions[0]/2)
  depth_x = x_pixels * cm2px[0] / 100
  # Calcular distancia en el eje Y
  depth_y = (dimensions[1] - center_coord[1]) * cm2px[1] / 100
  return [depth_x, depth_y]

def get_depth_from_map(coords, depth_map):
  import numpy as np
  depth_box = depth_map[coords[0]:coords[2], coords[1]:coords[3]]
  x_center = int((coords[2] - coords[0]) / 2)
  # return [np.min(depth_box), np.max(depth_box), np.mean(depth_box), depth_map[x_center, coords[3]-1]]
  return np.mean(depth_box)

In [0]:
import glob

names_boxes = glob.glob('/content/boxes_txt_coco/*')
names_depth = glob.glob('/content/depth_npy/*')
names_boxes = sorted(names_boxes)
names_depth = sorted(names_depth)

In [0]:
len(names_boxes)
print(names_boxes)
print(names_depth)

['/content/boxes_txt_coco/0000005.txt', '/content/boxes_txt_coco/0000006.txt', '/content/boxes_txt_coco/0000007.txt', '/content/boxes_txt_coco/0000008.txt', '/content/boxes_txt_coco/0000009.txt', '/content/boxes_txt_coco/0000010.txt', '/content/boxes_txt_coco/0000011.txt', '/content/boxes_txt_coco/0000012.txt', '/content/boxes_txt_coco/0000013.txt', '/content/boxes_txt_coco/0000014.txt', '/content/boxes_txt_coco/0000015.txt', '/content/boxes_txt_coco/0000016.txt', '/content/boxes_txt_coco/0000017.txt', '/content/boxes_txt_coco/0000018.txt', '/content/boxes_txt_coco/0000019.txt', '/content/boxes_txt_coco/0000020.txt', '/content/boxes_txt_coco/0000021.txt', '/content/boxes_txt_coco/0000022.txt', '/content/boxes_txt_coco/0000023.txt', '/content/boxes_txt_coco/0000024.txt', '/content/boxes_txt_coco/0000025.txt', '/content/boxes_txt_coco/0000026.txt', '/content/boxes_txt_coco/0000027.txt', '/content/boxes_txt_coco/0000028.txt', '/content/boxes_txt_coco/0000029.txt', '/content/boxes_txt_coco

In [0]:
import numpy as np

TM = np.array([[-5.97308814e-02, -7.27382808e-01,  1.62888095e+02],
               [-7.13056107e-03, -2.83119298e+00,  5.37303600e+02],
               [ 1.85065755e-06, -5.76037078e-03,  1.00000000e+00]])
px2cm = [16.0682, 16.0321]

h, w, l = 1.66, 1.68, 4.31
angle = -1.57


for frame in range(len(names_boxes)):

  # Loading depth
  depth = np.load(names_depth[frame]) * 80
  # Loading detections
  f = open(names_boxes[frame], "r")
  detections = f.readlines()
  f.close()

  # Generate object dictionary
  objects = {}
  for obj, i in zip(detections, range(len(detections))):
    coords = obj.split()
    objects[i] = np.array(coords).astype(np.float)

  # height, width, channels = img_original.shape
  width, height = 250, 500
  bev_vectors = {}
  depth_vectors = {}
  for i in objects:
    bev_vectors[i] = np.asarray(get_depth_vector(get_pixel_vector(TM, objects[i]), px2cm, [width,height])) #* np.asarray(px2cm)
    depth_vectors[i] = np.asarray(get_depth_from_map(objects[i].astype(int), depth.T))

  new_file = open("/content/detections_kitti_system/" + names_boxes[frame][24:], "w")
  # Generar detecciones (targets)
  for i in range(len(objects)):
    x = round(bev_vectors[i][0], 2)
    y = 1.73
    z = round(float(depth_vectors[i]), 2)
    if(z >= 25):
      z += 4.0
      if(x < 0):
        x = round(x - 0.9, 2)
      else:
        x = round(x + 0.9, 2)
    target = ['car', -1, -1, -1, objects[i][0], objects[i][1], objects[i][2], objects[i][3], h, w, l, x, y, z, angle, 0.9]
    new_file.write( str(target).replace(",", "")[1:-1] + '\n' )
  # print(z)
    
  new_file.close()
print('Frames: {0}'.format(frame+1))


Frames: 827


In [0]:
import os

file_name = 'detections_kitti_system'

# Generate Compressed File (file.zip)
import zipfile
fantasy_zip = zipfile.ZipFile('/content/' + file_name + '.zip', 'w') # Nombre del archivo comprimido
for folder, subfolders, files in os.walk('/content/' + file_name):
  for file in files:
    if file.endswith(''): # Extensión de los archivos a comprimir
      fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content/' + file_name), compress_type = zipfile.ZIP_DEFLATED)
fantasy_zip.close()
print('Compressed File generated (' + file_name + '.zip)!')

Compressed File generated (detections_kitti_system.zip)!


In [0]:
!cp /content/detections_kitti_system.zip '/gdrive/My Drive/'